На момент окончания работы с ноутбуком (29.04.2020 18:45) у меня тестовые письма определяются с точностью до наоборот: якобы первые два письма НЕспам, а последнее - спам

In [47]:
import pandas as pd
import re
import time

pA = 0    #P of SPAM
pNotA = 0 

cntA = 0    #qnt of SPAM letters
cntNotA = 0

dctA = {}
dctNotA = {}

SPAM = 1
NOT_SPAM = 0

df_en = pd.read_csv('data/words_alpha.txt', header=None) #external english words
words_en = df_en.to_dict('dict')[0]

len_A = 0
len_NotA = 0

In [48]:
# string transforming
def replace_symb(text):
    txt = str(text).lower()
    txt = re.sub('\.+|\,+|\_+|\!+|\?+|\;+|\:+|\"+', ' ', txt)
    txt = re.sub('[^0-9a-z ]', '', txt) #only digits, symbols, spaces
    return txt

# fill one dictionary
def fill_dict(p_dict, p_word):
    if p_word not in p_dict.keys():
        p_dict[p_word] = 1
    else:
        p_dict[p_word] += 1
        
# fill dictionaries
def calculate_word_frequencies(body, label):
    #global words_en #external dict of english words
    txt = replace_symb(body)
    word_old = None
    excl_list = ['is', 'a', 'an', 'i', 'am', 'the', 'this', 'these', 'that', 'to', 'and', 'of']
    
    for word in txt.split():
        word = str(word).strip()
#         if word_old == word: #skip multiple entries of the word
#             continue
#         elif word not in words_en.values(): #если оставить здесь, будет очень долго
#             continue
#         word_old = word
            
        if word in excl_list:
            continue
        elif label == SPAM:
            fill_dict(dctA, word)
        elif label == NOT_SPAM:
            fill_dict(dctNotA, word)

Хочу попробовать использовать внешний датасет английских слов words_alpha.txt.

Беглый анализ слов обучающих НЕспам писем показывает "кривизну" этих самых слов.

Предположение такое: сравниваем слова из писем с внешним "словарем" слов, и, если есть совпадение, слово в локальный словарь (dctA/dctNotA) добавляем, иначе - пропускаем

In [49]:
#train
def train():
    global pA, pNotA
    global dctA, dctNotA
    global cntA, cntNotA
    global words_en #external list of english words
    global len_A, len_NotA
    
    dctA = {}
    dctNotA = {}
    cntA = 0
    cntNotA = 0

    train_data = pd.read_csv('data/spam_or_not_spam.csv').values.tolist() #external letters

#     train_data = [
#         [' hello this is chinese traditional 子 件 NUMBER世 o最有效的宣 鞣绞剑 您想 地NUMBER f人同一 r刻知道您的 v告 幔磕 想方便快捷的宣 髂 的企 i 幔 您想在商 i 中步步 紫 c 幔 您想一次 碛 在商 i客 幔 越 碓蕉嗟恼 查表明 w路直 n 成 未 i n方式的主流 而e mail ⑹蔷w上 i n最常用也是最 用的工具 其特 c v r效高 r格低 emarketer估 美 有NUMBER 的中等 模公司 常 用 子 件 m行 i n活 印 部分 v告媒 w平均回 率比 average response rate ranges NUMBER NUMBER NUMBER 普通横幅广告 banner ads NUMBER NUMBER 普通信件 direct mail NUMBER NUMBER 子 件 email 一 件出售 i全球 件地址 每NUMBER f NUMBERhk i全部NUMBER f NUMBERhk i香港 件地址 每NUMBER f NUMBERhk i香港全部NUMBER f NUMBERhk hyperlink free download i _ 赤 件地址 每NUMBER f NUMBERhk i _ 橙 部NUMBER f NUMBERhk hyperlink free download 件 l送 v告 件代理 l送 每NUMBER f NUMBERhk NUMBER f起 l 蚀_率NUMBER 以上 包周群 l 一周NUMBER f NUMBERhk 一周NUMBER f NUMBERhk 保 c w站日 l 量提升NUMBER以上 包月群 l 一月NUMBER f NUMBERhk 保 c w站日 l 量提升NUMBER以上 件 w NUMBER n套群 l 搜索 w路 l 衍 w NUMBERhk 你自己 碛幸惶淄晟频碾 子 件群 l 搜索 w路 l 严到y 有意者请来信联系 hyperlink yhzxNUMBER URL 二 搜索引擎 缘顷 超值服 眨 特惠 rNUMBER hk y河之星 湍 登 全球NUMBER 英文搜索引擎 mNUMBER 中文搜索引擎 你的 w站享 u全球 e mail地址在各地 的分 盗 香港 NUMBER hyperlink free download the sample _ 场 NUMBER hyperlink free download the sample 全球 NUMBER f y河之星 碛 韧馍 有效的email位址 y料 欤 可以根 客 舻男枰 在指定的地方 域 行 i e等 傩酝斗烹 子 件 v告 如 你想 某地NUMBER f人一天之 都知道你的 a品 y 那麽 v告 件投放就是你最好的解 q方案 它的 r格是所有 髅 r格中最低 但又是最有效的 c遇每 r每刻都等待著你 也 s你正是用了 拥 髅菏侄危 而打垮了所有的 κ帧ｚs快行 影桑 你的企 i 公司立即掌握商 i 髅降拿 v告e mail 是普通 麂n i所不能比 m的 m及付款方式 本公司 氖戮w路 v告多年 宣 髁χ任痪又 第四大 w路 v告商之一 我 有 ｉt的技 g人 t和高速 i的服 障到y 氖麓隧 工作 必 您的企 i提供最迅捷 最有效 最 m意的服 铡 需求客 粽 c 暨 y m 箱 hyperlink yhzxNUMBER URL 硇耪 注明您需要的服 张c m方式 我 快回 湍 港 澳 _地 付款方式 任何 家均可到 地 y行 r NUMBER NUMBER天到 ぃ 需到 地 y行 k理 r english benificiary customer wangjingjing a c bank bank of china mianyang branch a c no NUMBER NUMBER NUMBER NUMBER benificiary s tel no NUMBER NUMBER 中文 φ眨 收 款 人 王晶晶 收款 y行 中 y行 d 分行 NUMBER NUMBER NUMBER NUMBER 收款人 NUMBER NUMBER NUMBER NUMBER天到 ぃ 款到後我 ⒘⒓ 您提供最 m意服 铡 g迎 硇 黼 j m NUMBER NUMBER NUMBER y河之星 w路 y 有限公司 公司地址 中 大 d email hyperlink yhzxNUMBER URL j NUMBER NUMBER NUMBER 公司服 NUMBER福壕w路推 v w站建 o w路 名 w 寄存 功能 得 q空 g w l展等 r x低廉 服 g迎 硇 黼 m ', 
#                   1],
#         [' date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER from chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com i can t reproduce this error for me it is very repeatable like every time without fail this is the debug log of the pick happening NUMBER NUMBER NUMBER pick_it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER ftoc_pickmsgs NUMBER hit NUMBER NUMBER NUMBER marking NUMBER hits NUMBER NUMBER NUMBER tkerror syntax error in expression int note if i run the pick command by hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER hit that s where the NUMBER hit comes from obviously the version of nmh i m using is delta pick version pick nmh NUMBER NUMBER NUMBER compiled on URL at sun mar NUMBER NUMBER NUMBER NUMBER ict NUMBER and the relevant part of my mh_profile delta mhparam pick seq sel list since the pick command works the sequence actually both of them the one that s explicit on the command line from the search popup and the one that comes from mh_profile do get created kre ps this is still using the version of the code form a day ago i haven t been able to reach the cvs repository today local routing issue i think _______________________________________________ exmh workers mailing list exmh workers URL URL ',
#         0]
#     ]
    
    print('train: fill dictionaries...')
    for i in range(len(train_data)):
        if train_data[i][1] == SPAM:
            cntA += 1
        else:
            cntNotA += 1
        calculate_word_frequencies(train_data[i][0], train_data[i][1])

    print('train: update dctA...')
    for key in dctA.keys():
        if key not in words_en.values():
            dctA[key] = 0 # обнуляем, имитируем удаление
            len_A += 1

    print('train: update dctNotA...')
    for key in dctNotA.keys():
        if key not in words_en.values():
            dctNotA[key] = 0 # обнуляем, имитируем удаление
            len_NotA += 1
    
    pA    = cntA / (cntA+cntNotA)
    pNotA = 1 - pA
    
    len_A = len(dctA) - len_A
    len_NotA = len(dctNotA) - len_NotA

#     print('SPAM dict:\n', dctA)
#     print('NOT_SPAM dict:\n', dctNotA)
    print('cntA    =', cntA, ', pA =', pA, ', len_A =', len_A)
    print('cntNotA =', cntNotA, ', pNotA =', pNotA, ', len_NotA =', len_NotA)

In [50]:
%%time
train()
# cntA    =  500 , pA    = 0.16666666666666666 , len(dctA)    = 14746
# cntNotA = 2500 , pNotA = 0.83333333333333340 , len(dctNotA) = 24987

# cntA    = 500 , pA = 0.16666666666666666 , len_A = 14736
# cntNotA = 2500 , pNotA = 0.8333333333333334 , len_NotA = 24771

# cntA    = 500 , pA = 0.16666666666666666 , len_A = 8036
# cntNotA = 2500 , pNotA = 0.8333333333333334 , len_NotA = 18550

# cntA    = 500 , pA = 0.16666666666666666 , len_A = 8033
# cntNotA = 2500 , pNotA = 0.8333333333333334 , len_NotA = 18547

train: fill dictionaries...
train: update dctA...
train: update dctNotA...
cntA    = 500 , pA = 0.16666666666666666 , len_A = 8033
cntNotA = 2500 , pNotA = 0.8333333333333334 , len_NotA = 18547
Wall time: 5min 31s


In [51]:
# get dictionary
def get_dict(label):
    if label == SPAM:
        return dctA
    else:
        return dctNotA
    
# one word
def calculate_P_Bi_A(word, label):
    global len_A, len_NotA
    d = get_dict(label)
    N = 0
    if label == SPAM:
        N = len_A
    elif label == NOT_SPAM:
        N = len_NotA
    qnt = d.get(word, 0) + 1
#     print(label, ': len_A =', len_A, ', len_NotA =', len_NotA)
#     print(label, ': word =', word, ', qnt =', qnt, ', N = ', N)
    p = qnt / N
    if p > 1: return 1 #некоторых слов так мног, что p > 1
    return p

In [52]:
word = 'mraaa'
print(calculate_P_Bi_A(word, SPAM))
print(calculate_P_Bi_A(word, NOT_SPAM))

0.00012448649321548613
5.391707553782283e-05


In [62]:
# whole sentence
def calculate_P_B_A(text, label):
    P = 1
    txt = replace_symb(text)
    for word in txt.split():
        prb = calculate_P_Bi_A(word, label)
        P *= prb
#         print(label,': word =', word, ', prb =', round(prb,6), ', P =', P)
    return P

In [63]:
%%time
text = 'Hi, My name is Warren E. Buffett an American business magnate, investor and philanthropist. am the most successful investor in the world. I believe strongly in‘giving while living’ I had one idea that never changed in my mind? that you should use your wealth to help people and i have decided to give {$1,500,000.00} One Million Five Hundred Thousand United Dollars, to randomly selected individuals worldwide. On receipt of this email, you should count yourself as the lucky individual. Your email address was chosen online while searching at random. Kindly get back to me at your earliest convenience before i travel to japan for my treatment , so I know your email address is valid. Thank you for accepting our offer, we are indeed grateful You Can Google my name for more information: God bless you. Best Regard Mr.Warren E. Buffett Billionaire investor !' 
# # text = 'Hi, My name is Warren E. Buffett an American business magnate, investor and philanthropist. am the most successful investor in the world. I believe strongly in‘giving while living’ ' 
# text = "Hi guys I want to build a website like REDACTED and I wanted to get your perspective of whether that site is good from the users' perspective before I go ahead and build something similar. I think that the design of the site is very modern and nice but I am not sure how people would react to a similar site? I look forward to your feedback. Many thanks!"
print(calculate_P_B_A(text, SPAM))
print(calculate_P_B_A(text, NOT_SPAM))

1 : word = hi , prb = 0.003112 , P = 0.003112162330387153
1 : word = my , prb = 0.069588 , P = 0.00021656899572842256
1 : word = name , prb = 0.039462 , P = 8.546292997125601e-06
1 : word = is , prb = 0.000124 , P = 1.0638980452042326e-09
1 : word = warren , prb = 0.000249 , P = 2.648818735725713e-13
1 : word = e , prb = 0.068841 , P = 1.8234741203240625e-14
1 : word = buffett , prb = 0.000124 , P = 2.2699789870833592e-18
1 : word = an , prb = 0.000124 , P = 2.8258172377484867e-22
1 : word = american , prb = 0.006473 , P = 1.8292356076549396e-24
1 : word = business , prb = 0.047056 , P = 8.607631765138394e-26
1 : word = magnate , prb = 0.000124 , P = 1.0715338933323034e-29
1 : word = investor , prb = 0.001618 , P = 1.7340894576521777e-32
1 : word = and , prb = 0.000124 , P = 2.1587071550506384e-36
1 : word = philanthropist , prb = 0.000124 , P = 2.6872988361143267e-40
1 : word = am , prb = 0.000124 , P = 3.345324083299299e-44
1 : word = the , prb = 0.000124 , P = 4.164476637992406e-48


0 : word = email , prb = 0.034938 , P = 3.537760221698592e-226
0 : word = address , prb = 0.007171 , P = 2.53691761193677e-228
0 : word = was , prb = 0.107726 , P = 2.732927906750238e-229
0 : word = chosen , prb = 0.000809 , P = 2.210272205815149e-232
0 : word = online , prb = 0.004691 , P = 1.0367912972767455e-234
0 : word = while , prb = 0.01666 , P = 1.727333319989833e-236
0 : word = searching , prb = 0.001456 , P = 2.5145845495080334e-239
0 : word = at , prb = 0.120828 , P = 3.0383264007373174e-240
0 : word = random , prb = 0.002103 , P = 6.388889288227497e-243
0 : word = kindly , prb = 0.000162 , P = 1.0334106790684473e-246
0 : word = get , prb = 0.063029 , P = 6.513490504291879e-248
0 : word = back , prb = 0.01666 , P = 1.0851720309625226e-249
0 : word = to , prb = 5.4e-05 , P = 5.850930236493894e-254
0 : word = me , prb = 0.044805 , P = 2.6215145449541306e-255
0 : word = at , prb = 0.120828 , P = 3.167527953438403e-256
0 : word = your , prb = 0.062921 , P = 1.9930474587063224e-2

In [55]:
# classify
def classify(email):
    global pA, pNotA
    P_B_A_SPAM = calculate_P_B_A(email, SPAM)
    P_B_A_notSPAM = calculate_P_B_A(email, NOT_SPAM)
    print('len(dctA)    =',len(dctA),', pA    =',pA,', P_B_A_SPAM =',P_B_A_SPAM,', P_SPAM =',pA * P_B_A_SPAM)
    print('len(dctNotA) =',len(dctNotA),', pNotA =',pNotA,', P_B_A_notSPAM =',P_B_A_notSPAM,', P_notSPAM =',pNotA * P_B_A_notSPAM)
    if pA * P_B_A_SPAM > pNotA * P_B_A_notSPAM:
        return True
    else:
        return False

### 2.3.1 Вопросы к заданию
Проверьте эти сообщения. Что говорит наш классификатор спама?

In [56]:
email = 'Hi, My name is Warren E. Buffett an American business magnate, investor and philanthropist. am the most successful investor in the world. I believe strongly in‘giving while living’ I had one idea that never changed in my mind? that you should use your wealth to help people and i have decided to give {$1,500,000.00} One Million Five Hundred Thousand United Dollars, to randomly selected individuals worldwide. On receipt of this email, you should count yourself as the lucky individual. Your email address was chosen online while searching at random. Kindly get back to me at your earliest convenience before i travel to japan for my treatment , so I know your email address is valid. Thank you for accepting our offer, we are indeed grateful You Can Google my name for more information: God bless you. Best Regard Mr.Warren E. Buffett Billionaire investor !'
# email = 'get NUMBER free vhs or dvds click'
classify(email)

len(dctA)    = 14724 , pA    = 0.16666666666666666 , P_B_A_SPAM = 0.0 , P_SPAM = 0.0
len(dctNotA) = 24759 , pNotA = 0.8333333333333334 , P_B_A_notSPAM = 0.0 , P_notSPAM = 0.0


False

In [57]:
email = "Hi guys I want to build a website like REDACTED and I wanted to get your perspective of whether that site is good from the users' perspective before I go ahead and build something similar. I think that the design of the site is very modern and nice but I am not sure how people would react to a similar site? I look forward to your feedback. Many thanks!"
classify(email)

len(dctA)    = 14724 , pA    = 0.16666666666666666 , P_B_A_SPAM = 9.54622312618996e-198 , P_SPAM = 1.5910371876983265e-198
len(dctNotA) = 24759 , pNotA = 0.8333333333333334 , P_B_A_notSPAM = 1.5491896496852615e-195 , P_notSPAM = 1.290991374737718e-195


False

In [58]:
email = 'As a result of your application for the position of Data Engineer, I would like to invite you to attend an interview on May 30, at 9 a.m. at our office in Washington, DC. You will have an interview with the department manager, Moris Peterson. The interview will last about 45 minutes. If the date or time of the interview is inconvenient, please contact me by phone or email to arrange another appointment. We look forward to seeing you.'
classify(email)

len(dctA)    = 14724 , pA    = 0.16666666666666666 , P_B_A_SPAM = 1.4880960584092581e-198 , P_SPAM = 2.480160097348763e-199
len(dctNotA) = 24759 , pNotA = 0.8333333333333334 , P_B_A_notSPAM = 1.2844957571343815e-201 , P_notSPAM = 1.070413130945318e-201


True

In [59]:
print('===== SPAM\n', 'len(dctA) =', len(dctA),'\n dict:\n', dctA)

===== SPAM
 len(dctA) = 14724 
 dict:
 {'save': 112, 'up': 283, 'number': 7367, 'on': 936, 'life': 164, 'insurance': 105, 'why': 68, 'spend': 9, 'more': 451, 'than': 152, 'you': 3512, 'have': 822, 'quote': 34, 'savings': 28, 'ensuring': 6, 'your': 1896, 'family': 70, 's': 767, 'financial': 106, 'security': 86, 'very': 114, 'important': 56, 'makes': 22, 'buying': 26, 'simple': 110, 'affordable': 13, 'we': 850, 'provide': 72, 'free': 717, 'access': 105, 'best': 157, 'companies': 75, 'lowest': 38, 'rates': 72, 'fast': 31, 'easy': 112, 'saves': 2, 'money': 548, 'let': 71, 'us': 336, 'help': 170, 'get': 430, 'started': 54, 'with': 1008, 'values': 4, 'in': 2152, 'country': 107, 'new': 296, 'coverage': 13, 'can': 547, 'hundreds': 53, 'or': 976, 'even': 100, 'thousands': 94, 'dollars': 133, 'by': 634, 'requesting': 19, 'from': 881, 'lifequote': 0, 'our': 770, 'service': 122, 'will': 861, 'take': 136, 'less': 54, 'minutes': 33, 'complete': 93, 'shop': 28, 'compare': 27, 'all': 605, 'types': 7, 

In [60]:
print('===== NOT_SPAM\n', 'len(dctNotA) =', len(dctNotA),'dict:\n', dctNotA)

===== NOT_SPAM
 len(dctNotA) = 24759 dict:
 {'date': 839, 'wed': 143, 'number': 26556, 'aug': 211, 'from': 2573, 'chris': 151, 'garrigues': 0, 'cwg': 0, 'dated': 26, 'numberfanumberd': 0, 'deepeddy': 0, 'com': 392, 'message': 703, 'id': 73, 'tmda': 0, 'vircio': 0, 'can': 1870, 't': 2933, 'reproduce': 12, 'error': 213, 'for': 4716, 'me': 830, 'it': 6069, 'very': 416, 'repeatable': 1, 'like': 1071, 'every': 277, 'time': 933, 'without': 286, 'fail': 29, 'debug': 35, 'log': 213, 'pick': 91, 'happening': 15, 'exec': 15, 'inbox': 0, 'list': 1784, 'lbrace': 0, 'subject': 392, 'ftp': 0, 'rbrace': 0, 'sequence': 78, 'mercury': 7, 'ftoc': 0, 'pickmsgs': 0, 'hit': 133, 'marking': 6, 'hits': 31, 'tkerror': 0, 'syntax': 20, 'in': 7340, 'expression': 24, 'int': 51, 'note': 69, 'if': 1938, 'run': 383, 'command': 92, 'by': 2038, 'hand': 113, 'delta': 17, 's': 4150, 'where': 493, 'comes': 78, 'obviously': 44, 'version': 266, 'nmh': 0, 'm': 896, 'using': 502, 'compiled': 37, 'on': 4097, 'url': 0, 'at': 

In [61]:
key = 'number'
print(dctA[key], dctNotA[key])

7367 26556
